In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/MBX/quip_interface/test


In [6]:
from collections import OrderedDict

In [7]:
version_list=["mbx_pbc","mbx_pbc_no_ww_www_co2ww","mbx_pbc_no_co2w"]

In [8]:
dict_version_model_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [9]:
dict_version_model_energy=OrderedDict([(version,np.NaN) for version in version_list])

In [10]:
dict_version_fortran_model_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [11]:
dict_version_fortran_model_energy=OrderedDict([(version,np.NaN) for version in version_list])

In [49]:
dict_version_single_point_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [50]:
dict_version_single_point_energy=OrderedDict([(version,np.NaN) for version in version_list])

In [12]:
this_root="/home/es732/programs/test_genice/CO2/sI/"

In [13]:
this_struct="sI_co2_cell2x2x2_rand0_ase.xyz"
this_at_tmp=quippy.Atoms(this_root+this_struct)
dict_pot=mbx_functions.create_mbx_potential_variables_co2_h2o(this_at_tmp)

In [14]:
np.sum(this_at_tmp.numbers==8)-64*2+64

432

In [15]:
dict_pot

{'n_monomers_types': '368 OHH 64 COO', 'nmon': 432}

## full MBX

In [21]:
this_version=version_list[0]
print this_version

mbx_pbc


In [22]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [23]:
at= this_at_tmp.copy()

In [24]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[ 1.47201270e+02  2.80973830e-02  1.46323351e-02]
 [ 2.80973830e-02  1.46872093e+02 -1.29849463e-01]
 [ 1.46323351e-02 -1.29849463e-01  4.13338055e+02]]


In [25]:
pot_MBX.calc(at,energy=True,virial=True)
print at.energy
print np.array( at.params["virial"])-dict_version_model_virial[this_version]

-229.340390325
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [26]:
dict_version_model_energy[this_version] =at.params["energy"]


In [27]:
print  dict_version_model_virial[this_version]/quippy.units.KCAL_MOL

[[ 3.39454195e+03  6.47941051e-01  3.37429665e-01]
 [ 6.47941051e-01  3.38695095e+03 -2.99439979e+00]
 [ 3.37429665e-01 -2.99439979e+00  9.53180205e+03]]


In [28]:
dict_version_fortran_model_virial["mbx_pbc"]=np.array([[3394.5391542125399 ,0.64789637185251880, 0.33740238917508236],
                                                       [0.64789637185251880,  3386.9480678837554, -2.9944082890992618],
                                                       [0.33740238917508236, -2.9944082890992618, 9531.7992736620254]])*quippy.units.KCAL_MOL

In [29]:
dict_version_fortran_model_energy["mbx_pbc"]=-5288.7150127307032*quippy.units.KCAL_MOL

In [52]:
dict_version_single_point_virial["mbx_pbc"]=np.array([[ 3.394541952877991e+03 , 6.479410507076393e-01 , 3.374296652847555e-01], 
                                                      [6.479410507076393e-01 , 3.386950951965182e+03 , -2.994399786073444e+00] ,
                                                      [3.374296652847555e-01 , -2.994399786073444e+00 , 9.531802052230920e+03]])*quippy.units.KCAL_MOL

In [53]:
dict_version_single_point_energy["mbx_pbc"]=-5288.72*quippy.units.KCAL_MOL

## fortran vs. python

In [55]:
(dict_version_fortran_model_virial["mbx_pbc"]-dict_version_model_virial["mbx_pbc"])/dict_version_model_virial["mbx_pbc"]

array([[-8.24460410e-07, -6.89551234e-05, -8.08349487e-05],
       [-6.89551234e-05, -8.51527367e-07,  2.83964281e-06],
       [-8.08349487e-05,  2.83964281e-06, -2.91505098e-07]])

In [56]:
(dict_version_fortran_model_energy["mbx_pbc"]-dict_version_model_energy["mbx_pbc"])/dict_version_model_energy["mbx_pbc"]

-1.3134418424669002e-08

## fortran vs. single_point

In [57]:
(dict_version_fortran_model_virial["mbx_pbc"]-dict_version_single_point_virial ["mbx_pbc"])/dict_version_single_point_virial["mbx_pbc"]

array([[-8.24460410e-07, -6.89551234e-05, -8.08349487e-05],
       [-6.89551234e-05, -8.51527367e-07,  2.83964281e-06],
       [-8.08349487e-05,  2.83964281e-06, -2.91505098e-07]])

In [58]:
(dict_version_fortran_model_energy["mbx_pbc"]-dict_version_single_point_energy["mbx_pbc"])/dict_version_single_point_energy["mbx_pbc"]

-9.430011982614963e-07

## diagonal_virial=F, no PIPs except mw 2B

In [32]:
this_version=version_list[1]
print this_version

mbx_pbc_no_ww_www_co2ww


In [33]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [34]:
at= this_at_tmp.copy()

In [35]:
pot_MBX.calc(at,virial=True,energy=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
dict_version_model_energy[this_version] =  at.params["energy"]

print dict_version_model_virial[this_version]

[[-5.26705602e+02  2.40198305e-01  4.29459541e-01]
 [ 2.40198305e-01 -5.27674169e+02 -1.98667968e+00]
 [ 4.29459541e-01 -1.98667968e+00 -2.59694841e+02]]


In [36]:
dict_version_fortran_model_energy["mbx_pbc_no_ww_www_co2ww"]=-10299.854656551084*quippy.units.KCAL_MOL

In [37]:
dict_version_fortran_model_virial["mbx_pbc_no_ww_www_co2ww"]=np.array([[-12146.117419954695,5.5390551892580904,9.9035525087171532],
                                                                       [5.5390551892580904,-12168.453235692657,-45.813889630451854],
                                                                       [9.9035525087171532,-45.813889630451854,-5988.7029654433718]])*quippy.units.KCAL_MOL

In [63]:
dict_version_single_point_virial["mbx_pbc_no_ww_www_co2ww"]=np.array([[-1.214611981198882e+04 , 5.539104534173878e+00 , 9.903572358641975e+00 ],
                                                                      [ 5.539104534173878e+00 , -1.216845549786064e+04 , -4.581392203901115e+01 ], 
                                                                      [9.903572358641975e+00 , -4.581392203901115e+01 , -5.988705355078668e+03 ]])*quippy.units.KCAL_MOL

In [64]:
dict_version_single_point_energy["mbx_pbc_no_ww_www_co2ww"]=-10299.9*quippy.units.KCAL_MOL

## fortran vs. python

In [65]:
(dict_version_fortran_model_energy["mbx_pbc_no_ww_www_co2ww"]-dict_version_model_energy["mbx_pbc_no_ww_www_co2ww"])/dict_version_model_energy["mbx_pbc_no_ww_www_co2ww"]

-1.9778607582625518e-07

In [66]:
(dict_version_fortran_model_virial["mbx_pbc_no_ww_www_co2ww"]-dict_version_model_virial["mbx_pbc_no_ww_www_co2ww"])/dict_version_model_virial["mbx_pbc_no_ww_www_co2ww"]

array([[-1.96938130e-07, -8.90846444e-06, -2.00431966e-06],
       [-8.90846444e-06, -1.85904282e-07, -7.07395435e-07],
       [-2.00431966e-06, -7.07395435e-07, -3.99023688e-07]])

## fortran vs. single_point

In [67]:
(dict_version_fortran_model_virial["mbx_pbc_no_ww_www_co2ww"]-dict_version_single_point_virial ["mbx_pbc_no_ww_www_co2ww"])/dict_version_single_point_virial["mbx_pbc_no_ww_www_co2ww"]

array([[-1.96938130e-07, -8.90846444e-06, -2.00431966e-06],
       [-8.90846444e-06, -1.85904282e-07, -7.07395435e-07],
       [-2.00431966e-06, -7.07395435e-07, -3.99023688e-07]])

In [68]:
(dict_version_fortran_model_energy["mbx_pbc_no_ww_www_co2ww"]-dict_version_single_point_energy["mbx_pbc_no_ww_www_co2ww"])/dict_version_single_point_energy["mbx_pbc_no_ww_www_co2ww"]

-4.402319334764841e-06

## full MBX - no 2B CO2 - water PIP

In [40]:
this_version=version_list[2]
print this_version

mbx_pbc_no_co2w


In [41]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [42]:
at= this_at_tmp.copy()

In [43]:
pot_MBX.calc(at,virial=True,energy=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
dict_version_model_energy[this_version] =at.params["energy"]

print dict_version_model_virial[this_version]

[[ 1.26240897e+02  1.17775182e-01  3.99142492e-01]
 [ 1.17775182e-01  1.25875417e+02 -3.97774287e-01]
 [ 3.99142492e-01 -3.97774287e-01  3.17334846e+02]]


In [44]:
dict_version_fortran_model_energy["mbx_pbc_no_co2w"]=-5385.7929701596913*quippy.units.KCAL_MOL

In [45]:
dict_version_fortran_model_virial["mbx_pbc_no_co2w"]=np.array([[2911.1819970827237,        2.7159200409817572,        9.2044141214315971],
                                                               [2.7159200409817572,        2902.7537707302527,       -9.1729030536742542],
                                                               [9.2044141214315971,       -9.1729030536742542,        7317.9131847538456]])*quippy.units.KCAL_MOL

In [59]:
dict_version_single_point_virial["mbx_pbc_no_co2w"]=np.array([[2.911184274803405e+03 , 2.715960239967526e+00, 9.204444597340984e+00] , 
                                                              [2.715960239967526e+00 , 2.902756092713413e+03 , -9.172893052672137e+00], 
                                                              [9.204444597340984e+00 , -9.172893052672137e+00 , 7.317915461213778e+03 ]])*quippy.units.KCAL_MOL

In [60]:
dict_version_single_point_energy["mbx_pbc_no_co2w"]=-5385.79*quippy.units.KCAL_MOL

## fortran vs. python

In [46]:
(dict_version_fortran_model_energy["mbx_pbc_no_co2w"]-dict_version_model_energy["mbx_pbc_no_co2w"])/dict_version_model_energy["mbx_pbc_no_co2w"]

-3.076269923878184e-08

In [47]:
(dict_version_fortran_model_virial["mbx_pbc_no_co2w"]-dict_version_model_virial["mbx_pbc_no_co2w"])/dict_version_model_virial["mbx_pbc_no_co2w"]

array([[-7.82403471e-07, -1.48010214e-05, -3.31099927e-06],
       [-1.48010214e-05, -7.99923619e-07,  1.09027785e-06],
       [-3.31099927e-06,  1.09027785e-06, -3.11080382e-07]])

In [48]:
(dict_version_fortran_model_virial["mbx_pbc_no_co2w"]-dict_version_model_virial["mbx_pbc_no_co2w"])

array([[-9.87713163e-05, -1.74319300e-06, -1.32156050e-06],
       [-1.74319300e-06, -1.00690719e-04, -4.33684495e-07],
       [-1.32156050e-06, -4.33684495e-07, -9.87166451e-05]])

## fortran vs. single_point

In [61]:
(dict_version_fortran_model_virial["mbx_pbc_no_co2w"]-dict_version_single_point_virial ["mbx_pbc_no_co2w"])/dict_version_single_point_virial["mbx_pbc_no_co2w"]

array([[-7.82403471e-07, -1.48010214e-05, -3.31099927e-06],
       [-1.48010214e-05, -7.99923620e-07,  1.09027785e-06],
       [-3.31099927e-06,  1.09027785e-06, -3.11080382e-07]])

In [62]:
(dict_version_fortran_model_energy["mbx_pbc_no_co2w"]-dict_version_single_point_energy["mbx_pbc_no_co2w"])/dict_version_single_point_energy["mbx_pbc_no_co2w"]

5.514807840536607e-07